In [71]:
from uuid import uuid3
from random import randint
import random
import pandas as pd

def create_username(fav_cuisine):
    fav_words = ["luver","connoisseur","eatz","finesser","fan"]
    need_words = ["cant_get_enough","cant_live_without","needs","only_eats"]
    names = ['Kathey', 'Silas', 'Cristina', 'Perry', 'Robbi', 'Florentino', 'Elliot', 'Raye',\
             'Evette', 'Risa', 'Lurline', 'Tashia', 'Danyel', 'Shaun', 'Luciano', 'Trang', \
             'Summer', 'Martin', 'China', 'Jacob', 'Alenta', 'Sule', 'Quentin', 'Anton']
    food_words = ["food","grub","eatz"]
    
    rand_num = ''.join(random.choice('0123456789') for i in range(3))
    if randint(0,1) == 0:
        return "{}_{}_{}".format(fav_cuisine.lower(),fav_words[randint(0,len(fav_words)-1)],rand_num)
    
    i,j,k = randint(0,len(names)-1),randint(0,len(need_words)-1),randint(0,len(food_words)-1)
    return "{}_{}_{}_{}_{}".format(names[i].lower(), need_words[j], fav_cuisine.lower(),food_words[k],rand_num)
    
def get_rating(x):
    if x["cuisine"] == x["fav_cuisine"]:
        return 5
    elif x["cuisine"] == x["least_fav_cuisine"]:
        return 1
    else:
        return randint(1,5)

def createUserReviews(num_users,review_ratio,recipe_df):
    users_df = pd.DataFrame(columns=["username","favorite_cusine","least_fav_cuisine"])
    reviews_df = pd.DataFrame(columns=["username","recipe_id","rating"])
    cuisines = recipe_df.cuisine.unique()

    for i in range(num_users):
        fav_cuisine_index = randint(0,len(cuisines)-1)
        least_fav_cuisine_index = randint(0,len(cuisines)-1)
        while least_fav_cuisine_index == fav_cuisine_index:
            least_fav_cuisine_index = randint(0,len(cuisines)-1)
        fav_cuisine,least_fav_cuisine = cuisines[fav_cuisine_index],cuisines[least_fav_cuisine_index]
        username = create_username(fav_cuisine)
        user_data = [username,fav_cuisine,least_fav_cuisine]
        users_df.loc[i] = user_data
        
        for c in cuisines:
            mask = recipe_df["cuisine"] == c
            recipe_subset = recipe_df[mask]
            rows,cols = recipe_subset.shape
            to_be_reviewed = recipe_subset.sample(int(rows*review_ratio))
            to_be_reviewed["username"] = username
            to_be_reviewed["fav_cuisine"] = fav_cuisine
            to_be_reviewed["least_fav_cuisine"] = least_fav_cuisine
            to_be_reviewed["rating"] = to_be_reviewed.apply(get_rating,axis=1)
            reviewed = to_be_reviewed[["username","recipe_id","rating"]]
            reviews_df = reviews_df.append(reviewed)
            
    return users_df, reviews_df
    

In [75]:
path = '../data/synthetic/recipes/cuisine_size_10.csv'
recipe_df = pd.read_csv(path)
num_users = 20
review_ratio = .3
users_df, reviews_df = createUserReviews(num_users,review_ratio,recipe_df)  

users_df.to_csv('../data/synthetic/users/{}_users_{}_reviewratio.csv'.format(num_users,review_ratio))
reviews_df.to_csv('../data/synthetic/reviews/{}_users_{}_reviewratio.csv'.format(num_users,int(review_ratio*100)))